In [1]:
import jukebox
import torch as t
import librosa
import os
from IPython.display import Audio
from jukebox.make_models import make_vqvae, make_prior, MODELS, make_model
from jukebox.hparams import Hyperparams, setup_hparams
from jukebox.sample import sample_single_window, _sample, \
                           sample_partial_window, upsample
from jukebox.utils.dist_utils import setup_dist_from_mpi
from jukebox.utils.torch_utils import empty_cache
rank, local_rank, device = setup_dist_from_mpi()

Using cuda True


# Sample from the 5B or 1B Lyrics Model
5Bまたは1Bの歌詞モデルからのサンプル


In [7]:
model = "1b_lyrics_finetune_roma_11708" # or "1b_lyrics"     
hps = Hyperparams()
hps.sr = 44100
hps.n_samples = 3 if model=='5b_lyrics' else 8
hps.name = 'samples'
chunk_size = 16 if model=="5b_lyrics" else 32
max_batch_size = 3 if model=="5b_lyrics" else 16
hps.levels = 3
hps.hop_fraction = [.5,.5,.125]

vqvae, *priors = MODELS[model]
vqvae = make_vqvae(setup_hparams(vqvae, dict(sample_length = 1048576)), device)
top_prior = make_prior(setup_hparams(priors[-1], dict()), vqvae, device)



Restored from /root/.cache/jukebox/models/5b/vqvae.pth.tar
0: Loading vqvae in eval mode
【hps.jp_lyrics】 : False
Creating cond. autoregress with prior bins [79, 2048], 
dims [384, 6144], 
shift [ 0 79]
input shape 6528
input bins 2127
Self copy is False
using /workspace/jukebox/data/ids/v3_artist_ids.txt
Loading artist IDs from /workspace/jukebox/data/ids/v3_artist_ids.txt
Loading artist IDs from /workspace/jukebox/data/ids/v3_genre_ids.txt
【n_vocab】 : 79
Level:2, Cond downsample:None, Raw to tokens:128, Sample length:786432
Restored from /workspace/logs/finetuned_11708_20210711/checkpoint_epoch_24.pth.tar
0: Loading prior in eval mode


## Artist Embeddingを可視化

In [8]:
params = top_prior.y_emb.artist_emb.state_dict()
artist_emb_np = params['emb.weight'].to('cpu').detach().numpy().copy()

In [9]:
print(f"(artist, prior_width) : {artist_emb_np.shape}")

(artist, prior_width) : (7898, 2048)


In [10]:
import numpy as np
np.save('./weights_np/roma_artist_emb.npy', artist_emb_np)

In [6]:
artist_emb_np

array([[ 4.72012348e-03,  5.45738731e-03,  1.12937670e-02, ...,
         4.06365981e-03,  4.18949639e-03,  1.41272740e-03],
       [-2.93332938e-04,  2.04770570e-03, -1.31708002e-02, ...,
        -1.08278217e-03, -1.36694161e-03, -4.31381445e-03],
       [ 1.51319522e-02,  3.69301392e-03,  6.83256658e-03, ...,
         2.66497508e-02,  3.30203515e-03, -4.24456829e-03],
       ...,
       [ 1.10084107e-02,  1.13135790e-02, -8.07391573e-03, ...,
         2.60757282e-02, -7.91268796e-03, -9.48189478e-03],
       [-2.89823045e-04,  1.98168127e-04, -5.12999657e-04, ...,
         9.04048356e-05, -4.27678679e-05, -1.17053365e-04],
       [ 8.55198305e-05,  2.71445839e-04, -1.02780410e-04, ...,
         2.10192666e-04, -1.44701742e-04, -8.72013406e-05]], dtype=float32)

### T-SNEで可視化

In [7]:
import numpy as np
from sklearn.manifold import TSNE

In [8]:

artist_emb_tsne = TSNE(n_components=2).fit_transform(artist_emb_np)
print(artist_emb_tsne.shape)

(7898, 2)
